In [2]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

In [3]:
open FSharp.Data
open Plotly.NET

let trade_csv = "d:\\studies\\03_export_date_ticker_screenerid_gap_outcomes.csv"

let csv = studies.GapStudy.parseTradeOutcomes trade_csv

In [4]:
let outputSummary includeChart (summary:studies.Types.TradeSummary) = 

    printfn "%s: %i trades, %.2f%% win pct, %.2f%% avg gain, %.2f%% avg loss, %.2f avg gain/loss, %.2f EV" 
        summary.StrategyName summary.Total (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.EV

    match includeChart with
    | true ->
        let chart = 
            Chart.Histogram(
                X = summary.Gains,
                NBinsX = 40,
                Name = "Gain Distribution"
            )

        chart.Display() |> ignore
    | false -> ()

In [23]:
let filterNamePairs = [
        ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
        ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
        ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
        ("Gap ups", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp)
        ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 28)
        ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 29)
    ]
    
let describeOutcomes outcomes =
    
    let tradesGroupedByStrategy = outcomes |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    filterNamePairs
    |> Seq.iter (
        fun (filterName,filter) ->

            printfn $"Trade: {filterName}"
            tradesGroupedByStrategy
            |> Seq.iter (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    let summary = studies.Types.TradeSummary.create filteredTrades
                    outputSummary false summary
            )

            printfn ""
            printfn ""
    )
    
let highlightStrategies outcomes =
    
    let tradesGroupedByStrategy = outcomes |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy)
    
    let data =
        filterNamePairs
        |> Seq.map (fun (filterName, filter) ->
            tradesGroupedByStrategy
            |> Seq.map (fun (strategy, trades) ->
                let filteredTrades = trades |> Seq.filter filter
                let summary = studies.Types.TradeSummary.create filteredTrades
                (strategy, filterName, summary)
            )
        )
        |> Seq.concat

    let sortFunctions = [
        ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
        ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
        ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top 5 by: {sortLabel}"
        data
        |> Seq.sortByDescending (fun (strategy, filterName, summary) -> sortFunction summary)
        |> Seq.take 5
        |> Seq.iter (fun (strategy, filterName, summary) ->
            printfn $"Strategy: {strategy}, Filter: {filterName}: {sortFunction summary}"
        )
        printfn ""
        printfn ""
    )

Top 5 by: EV
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.822461023902737286555059309
Strategy: Buy and Hold, Filter: New Highs: 0.736784720611110804648903637
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5825484022408140426570786786
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5698105391586863628178409871
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.4783539464164832646621860451


Top 5 by: Win %
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5171658660638598474145238768
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5154989384288747346072186837
Strategy: Buy and Hold 10 bars, Filter: Gap ups - top gainers: 0.5131634819532908704883227176
Strategy: Buy and Hold 10 bars, Filter: Gap ups: 0.5081433224755700325732899023
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.507057546145494028230184582


Top 5 by: Avg Gain
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.2635946275090736548676078349
Strategy: Buy and Hold 5 

In [7]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

In [8]:
let studiesDirectory = "d:\\studies"
let signalPath = "d:\\studies\\02_export_date_ticker_screenerid_gap.csv"
let priceFunc = studies.DataHelpers.getPricesFromCsv studiesDirectory

let runStrategies strategies = async {
    let! signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc
    
    return! studies.Trading.runTrades signalsWithPrices strategies
}

In [9]:
let strategies = 
        [
            studies.TradingStrategies.buyAndHoldStrategy (Some 5)
            studies.TradingStrategies.buyAndHoldStrategy (Some 10)
            studies.TradingStrategies.buyAndHoldStrategy (Some 30)
            studies.TradingStrategies.buyAndHoldStrategy (Some 60)
            studies.TradingStrategies.buyAndHoldStrategy (Some 90)
            studies.TradingStrategies.buyAndHoldStrategy None
        ]

let outcomes = 
    strategies
    |> runStrategies
    |> Async.RunSynchronously

printfn "Finished running trades, generated:"
printfn $"{outcomes |> Seq.length} trade outcomes"

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Executing trades...
Finished running trades, generated:
137088 trade outcomes


In [17]:
outcomes |> Seq.head |> fun x -> $"{x.Strategy}: {x.Opened} @ {x.OpenPrice}, {x.Closed} @ {x.ClosePrice}: profit of {x.PercentGain}"

Buy and Hold 5 bars: 3/23/2022 12:00:00 AM @ 16.73, 3/30/2022 12:00:00 AM @ 19.57: profit of 16.975493126120741183502689780

In [24]:
outcomes |> highlightStrategies

Top 5 by: EV
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.822461023902737286555059309
Strategy: Buy and Hold, Filter: New Highs: 0.736784720611110804648903637
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5825484022408140426570786786
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5698105391586863628178409871
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.4783539464164832646621860451


Top 5 by: Win %
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5171658660638598474145238768
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5154989384288747346072186837
Strategy: Buy and Hold 10 bars, Filter: Gap ups - top gainers: 0.5131634819532908704883227176
Strategy: Buy and Hold 10 bars, Filter: Gap ups: 0.5081433224755700325732899023
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.507057546145494028230184582


Top 5 by: Avg Gain
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.2635946275090736548676078349
Strategy: Buy and Hold 5 